In [6]:
#| default_exp 10_msmarco-generated-entities

In [7]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [8]:
#| export
from tqdm.auto import tqdm
from dataclasses import dataclass
import json, re, scipy.sparse as sp, pickle, numpy as np, argparse, pandas as pd

In [5]:
#| export
def load_raw_txt(fname):
    ids, txt = list(), list()
    with open(fname, 'r') as file:
        for line in file:
            k,v = line[:-1].split("->", maxsplit=1)
            ids.append(int(k)); txt.append(v)
    return ids, txt
    

In [10]:
#| export
def select_entity(entities, entity_type='entity_canonical_category'):
    if entity_type == 'entity_canonical_category': 
        return [o.lower() for o in entities if len(o.split(' | ')) == 3]
    elif entity_type == 'entity': 
        return [o.split(' | ')[0].lower() for o in entities if len(o.split(' | ')) == 3]
    elif entity_type == 'canonical': 
        return [o.split(' | ')[1].lower() for o in entities if len(o.split(' | ')) == 3]
    elif entity_type == 'category': 
        return [o.split(' | ')[2].lower() for o in entities if len(o.split(' | ')) == 3]
    elif entity_type == 'entity_canonical': 
        return [' | '.join(o.split(' | ')[0:2]).lower() for o in entities if len(o.split(' | ')) == 3]
    else: 
        raise ValueError(f'Invalid entity type: {entity_type}')
    

In [13]:
#| export
def load_gpt_entities(fname, entity_type='entity_canonical_category'):
    with open(fname, 'rb') as file:
        generations = pickle.load(file)
        
    entities = dict()
    for k,v in tqdm(generations.items(), total=len(generations)):
        query = k.split('\t')[1].split('==::==')[0].split('|||')[1]
        entities.setdefault(query, []).extend(select_entity(v['Entities'], entity_type))
        
    return entities


In [14]:
#| export
def load_llama_entities(fname, entity_type='entity_canonical_category'):
    with open(fname, 'r') as file:
        generations = [json.loads(line) for line in file]

    entities = dict()
    pattern = r"<Entities>(.*?)</Entities>"
    for generation in tqdm(generations, total=len(generations)):
        entity = [entity.strip() for entity in re.findall(pattern, generation['gen'], re.DOTALL) if len(entity.strip().split(' | ')) == 3]
        entities.setdefault(int(generation['qid']), []).extend(select_entity(entity, entity_type))
        
    return entities
    

In [16]:
#| export
def load_data(data_dir, entity_dir, entity_type='entity_canonical_category', pretrained='gpt'):
    # training data
    trn_ids, trn_txt = load_raw_txt(f'{data_dir}/raw_data/train.raw.txt')
    if pretrained == 'gpt': 
        trn_entities = load_gpt_entities(f'{entity_dir}/trn_gpt_generations.pkl', entity_type=entity_type)
    elif pretrained == 'llama': 
        trn_entities = load_llama_entities(f'{entity_dir}/trn_llama_generations.txt', entity_type=entity_type)
    else: 
        raise ValueError(f'Invalid pretrained type: {pretrained}')
    trn_data = (trn_ids, trn_txt, trn_entities)

    # testing data
    tst_ids, tst_txt = load_raw_txt(f'{data_dir}/raw_data/test.raw.txt')
    if pretrained == 'gpt': 
        tst_entities = load_gpt_entities(f'{entity_dir}/dev_gpt_generations.pkl', entity_type=entity_type)
    elif pretrained == 'llama': 
        tst_entities = load_llama_entities(f'{entity_dir}/dev_llama_generations.txt', entity_type=entity_type)
    else: 
        raise ValueError(f'Invalid pretrained type: {pretrained}')
    tst_data = (tst_ids, tst_txt, tst_entities)

    # vocabulary creation
    entity_to_idx = dict()
    for v in trn_entities.values():
        for e in v: entity_to_idx.setdefault(e, len(entity_to_idx))
    for v in tst_entities.values():
        for e in v: entity_to_idx.setdefault(e, len(entity_to_idx))
        
    return trn_data, tst_data, entity_to_idx
        

In [17]:
#| export
def save_raw_txt(fname, ids, texts):
    with open(fname, 'w') as file:
        for k,v in tqdm(zip(ids, texts), total=len(ids)):
            file.write(f'{k}->{v}\n')
        

In [18]:
#| export
def construct_matrix(texts, entities, entity_to_idx):
    data, indices, indptr = [], [], [0]
    for qtxt in tqdm(texts):
        entity = entities.get(qtxt, [])
        data.extend([1] * len(entity))
        indices.extend([entity_to_idx[o] for o in entity])
        indptr.append(len(indices))
    return sp.csr_matrix((data, indices, indptr), shape=(len(texts), len(entity_to_idx)), dtype=np.int64)


In [19]:
#| export
@dataclass
class EntityInfo:
    ids: list
    txt: list
    trn_mat: sp.csr_matrix
    tst_mat: sp.csr_matrix
    lbl_mat: sp.csr_matrix

    @property
    def stats(self):
        n_trn, n_entity = self.trn_mat.shape
        n_tst, n_lbl = self.tst_mat.shape[0], self.lbl_mat.shape[0]
        
        def entity(mat):
            ent_p_dat = mat.getnnz(axis=1).mean()
            dat_p_ent = mat.getnnz(axis=0).mean()
            
            dat_no_ent = np.sum(mat.getnnz(axis=1) == 0)
            ent_no_dat = np.sum(mat.getnnz(axis=0) == 0)
            
            return {
                'Number of queries per entity': ent_p_dat, 
                'Number of entities per query': dat_p_ent, 
                'Number of queries without entity': dat_no_ent, 
                'Number of entities without query': ent_no_dat,
            }
    
        stats = [{'Number of queries': n_trn, 'Number of entities': n_entity, **entity(self.trn_mat)}] + \
        [{'Number of queries': n_tst, 'Number of entities': n_entity, **entity(self.tst_mat)}] + \
        [{'Number of queries': n_lbl, 'Number of entities': n_entity, **entity(self.lbl_mat)}]
        
        return pd.DataFrame(stats, index=['Train', 'Test', 'Labels'])
        

In [20]:
#| export 
def construct_msmarco_entities(data_dir:str, entity_dir:str, entity_type='entity_canonical_category', pretrained='gpt'):
    trn_data, tst_data, entity_to_idx = load_data(data_dir, entity_dir, entity_type=entity_type, pretrained=pretrained)

    trn_mat = construct_matrix(trn_data[1] if pretrained == 'gpt' else trn_data[0], trn_data[2], entity_to_idx)
    
    valid_idx = np.where(trn_mat.getnnz(axis=0) > 0)[0]
    trn_mat = trn_mat[:, valid_idx]

    entity_txt = [text for v, text in zip(valid_idx, sorted(entity_to_idx, key=lambda x: entity_to_idx[x])) if v]
    entity_ids = list(range(len(entity_txt)))

    tst_mat = construct_matrix(tst_data[1] if pretrained == 'gpt' else tst_data[0], tst_data[2], entity_to_idx)[:, valid_idx]
    lbl_mat = sp.csr_matrix((sp.load_npz(f'{data_dir}/trn_X_Y.npz').shape[1], len(entity_txt)), dtype=np.int64)

    return EntityInfo(entity_ids, entity_txt, trn_mat, tst_mat, lbl_mat)
    

In [26]:
#| export
def save_msmarco_entities(data_dir, pretrained, entity_type, entity_info):
    save_raw_txt(f'{data_dir}/raw_data/{pretrained}_{entity_type}.raw.txt', entity_info.ids, entity_info.txt)
    sp.save_npz(f'{data_dir}/{pretrained}_{entity_type}_trn_X_Y.npz', entity_info.trn_mat)
    sp.save_npz(f'{data_dir}/{pretrained}_{entity_type}_tst_X_Y.npz', entity_info.tst_mat)
    sp.save_npz(f'{data_dir}/{pretrained}_{entity_type}_lbl_X_Y.npz', entity_info.lbl_mat)
    

In [24]:
#| export
def construct_save_msmarco_entities(data_dir:str, entity_dir:str, entity_type='entity_canonical_category', 
                                    pretrained='gpt', do_save:bool=False):
    entity_info = construct_msmarco_entities(data_dir, entity_dir, entity_type, pretrained=pretrained)
    if do_save: save_msmarco_entities(data_dir, pretrained, entity_type, entity_info)
    return entity_info
    

In [25]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--entity_dir', type=str, required=True)
    parser.add_argument('--entity_type', type=str, required=True)
    parser.add_argument('--pretrained', action='store_true')
    parser.add_argument('--save', action='store_true')
    return parser.parse_args()
    

In [163]:
#| export
if __name__ == '__main__':
    args = parse_args()
    construct_save_msmarco_entities(args.data_dir, args.entity_dir, args.entity_type, args.pretrained, args.save)

In [161]:
data_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/XC"
entity_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/entities/gpt/"
entity_type = 'entity'
pretrained = 'gpt'
save = False

entity_info = construct_save_msmarco_entities(data_dir, entity_dir, entity_type=entity_type, pretrained=pretrained, save=save)

  0%|          | 0/532743 [00:00<?, ?it/s]

  0%|          | 0/34621 [00:00<?, ?it/s]

  0%|          | 0/502939 [00:00<?, ?it/s]

  0%|          | 0/6980 [00:00<?, ?it/s]

In [254]:
def get_entity_stats(entity_type, pretrained):
    data_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco-data/XC"
    entity_dir = f"/home/scai/phd/aiz218323/scratch/datasets/msmarco_entities/{pretrained}/"
    save = False
    
    entity_info = construct_save_msmarco_entities(data_dir, entity_dir, entity_type=entity_type, pretrained=pretrained, save=save)
    return entity_info.stats
    

In [167]:
get_entity_stats('entity', 'gpt')

  0%|          | 0/532743 [00:00<?, ?it/s]

  0%|          | 0/34621 [00:00<?, ?it/s]

  0%|          | 0/502939 [00:00<?, ?it/s]

  0%|          | 0/6980 [00:00<?, ?it/s]

,Number of queries,Number of entities,Number of queries per entity,Number of entities per query,Number of queries without entity,Number of entities without query
Train,502939,207109,1.567920,3.807502,28386,0
Test,6980,207109,0.763037,0.025716,3821,203538
Labels,8841823,207109,0.000000,0.000000,8841823,207109


In [255]:
get_entity_stats('entity', 'llama')

  0%|          | 0/532755 [00:00<?, ?it/s]

  0%|          | 0/59265 [00:00<?, ?it/s]

  0%|          | 0/502939 [00:00<?, ?it/s]

  0%|          | 0/6980 [00:00<?, ?it/s]

,Number of queries,Number of entities,Number of queries per entity,Number of entities per query,Number of queries without entity,Number of entities without query
Train,502939,200337,1.488896,3.737822,30245,0
Test,6980,200337,1.120774,0.039049,2047,195395
Labels,8841823,200337,0.000000,0.000000,8841823,200337
